In [1]:
cd "c:\Users\ecometto001\Documents\Personal\Tesis"

c:\Users\ecometto001\Documents\Personal\Tesis


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
plt.style.use("dark_background")
random_state=42

In [3]:
tesis = pd.read_csv("Datos/tesis_final.csv")
tesis = tesis[tesis["tpo_programa"] <= 12]
tesis["Intercept"] = 1

In [4]:
tesis_lag = tesis[["idPaciente", "Intercept", "TAS", "tpo_programa", "Adherencia", "Adherencia_Acumulada", "TAS_Media_Acumulada"]]
lagged_vars = ["Adherencia", "TAS", "Adherencia_Acumulada", "TAS_Media_Acumulada"]
for var in lagged_vars:
    for i in range(1, 4):
        tesis_lag[f"{var}_lag{i}"] = tesis_lag.groupby("idPaciente")[var].shift(i)

In [5]:
from statsmodels.discrete.discrete_model import Logit
fixed_effects = ["Intercept", "Adherencia_lag1", "Adherencia_Acumulada_lag2",  "TAS_lag1", "TAS_Media_Acumulada_lag2"]
lr = Logit(tesis_lag.dropna()["Adherencia"], tesis_lag.dropna()[fixed_effects], groups=tesis_lag["idPaciente"]).fit()
print(lr.summary())
# Las covariables para TAS no son significativas

Optimization terminated successfully.
         Current function value: 0.350268
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Adherencia   No. Observations:                 2240
Model:                          Logit   Df Residuals:                     2235
Method:                           MLE   Df Model:                            4
Date:                Wed, 11 Jan 2023   Pseudo R-squ.:                  0.2120
Time:                        18:02:26   Log-Likelihood:                -784.60
converged:                       True   LL-Null:                       -995.73
Covariance Type:            nonrobust   LLR p-value:                 4.298e-90
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -0.0842      0.867     -0.097      0.923      -1.

c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)


In [6]:
max_range = tesis["tpo_programa"].max()+1

# Modelos para cada tiempo con 1 variables rezagada

In [7]:
# Exogeneidad por tiempo con un solo lag
from statsmodels.discrete.discrete_model import Logit
tiempos = []
for i in range(1,max_range):
    tesis_aux = tesis_lag[tesis_lag["tpo_programa"] == i]
    print("\n", "*"*50, "Target time:", i, "*"*50, "\n")
    fixed_effects = ["Intercept", "Adherencia_lag1", "TAS_lag1"]
    lr = Logit(tesis_aux["Adherencia"], tesis_aux[fixed_effects], groups=tesis_aux["idPaciente"]).fit()
    print(lr.summary())
    if lr.pvalues["TAS_lag1"] < 0.05:
        tiempos.append(str(i))


 ************************************************** Target time: 1 ************************************************** 

Optimization terminated successfully.
         Current function value: 0.438335
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Adherencia   No. Observations:                  560
Model:                          Logit   Df Residuals:                      557
Method:                           MLE   Df Model:                            2
Date:                Wed, 11 Jan 2023   Pseudo R-squ.:                  0.1240
Time:                        18:02:26   Log-Likelihood:                -245.47
converged:                       True   LL-Null:                       -280.23
Covariance Type:            nonrobust   LLR p-value:                 8.035e-16
                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------

c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\st

In [8]:
print(f"De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos {', '.join(tiempos)}")

De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos 


# Modelos para cada tiempo con 1 variables rezagada y promedio acumulado

In [9]:
# Exogeneidad por tiempo con un solo lag
from statsmodels.discrete.discrete_model import Logit
tiempos = []
for i in range(2,max_range):
    tesis_aux = tesis_lag[tesis_lag["tpo_programa"] == i]
    print("\n", "*"*50, "Target time:", i, "*"*50, "\n")
    fixed_effects = ["Intercept", "Adherencia_lag1", "Adherencia_Acumulada_lag2", "TAS_lag1", "TAS_Media_Acumulada_lag2"]
    lr = Logit(tesis_aux["Adherencia"], tesis_aux[fixed_effects], groups=tesis_aux["idPaciente"]).fit()
    print(lr.summary())
    if (lr.pvalues["TAS_lag1"] < 0.05) or (lr.pvalues["TAS_Media_Acumulada_lag2"] < 0.05):
        tiempos.append(str(i))


 ************************************************** Target time: 2 ************************************************** 

Optimization terminated successfully.
         Current function value: 0.369976
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Adherencia   No. Observations:                  560
Model:                          Logit   Df Residuals:                      555
Method:                           MLE   Df Model:                            4
Date:                Wed, 11 Jan 2023   Pseudo R-squ.:                  0.2115
Time:                        18:02:27   Log-Likelihood:                -207.19
converged:                       True   LL-Null:                       -262.76
Covariance Type:            nonrobust   LLR p-value:                 4.131e-23
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------

c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\st

In [10]:
print(f"De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos {', '.join(tiempos)}")

De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos 


# Modelos para cada tiempo con 2 variables rezagadas y promedio acumulado

In [11]:
# Exogeneidad por tiempo con dos lag
from statsmodels.discrete.discrete_model import Logit
tiempos = []
for i in range(3,max_range):
    tesis_aux = tesis_lag[tesis_lag["tpo_programa"] == i]
    print("\n", "*"*50, "Target time:", i, "*"*50, "\n")
    fixed_effects = ["Intercept", "Adherencia_lag1", "Adherencia_lag2", "Adherencia_Acumulada_lag3", "TAS_lag1", "TAS_lag2", "TAS_Media_Acumulada_lag3"]
    lr = Logit(tesis_aux["Adherencia"], tesis_aux[fixed_effects], groups=tesis_aux["idPaciente"]).fit()
    print(lr.summary())
    if (lr.pvalues["TAS_lag1"] < 0.05) or (lr.pvalues["TAS_lag2"] < 0.05) or (lr.pvalues["TAS_Media_Acumulada_lag3"] < 0.05):
        tiempos.append(str(i))

c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)



 ************************************************** Target time: 3 ************************************************** 

Optimization terminated successfully.
         Current function value: 0.376204
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Adherencia   No. Observations:                  560
Model:                          Logit   Df Residuals:                      553
Method:                           MLE   Df Model:                            6
Date:                Wed, 11 Jan 2023   Pseudo R-squ.:                  0.1887
Time:                        18:02:27   Log-Likelihood:                -210.67
converged:                       True   LL-Null:                       -259.69
Covariance Type:            nonrobust   LLR p-value:                 6.476e-19
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------

c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['groups']
  warnings.warn(msg, ValueWarning)
c:\Users\ecometto001\AppData\Local\Programs\Python\Python39\lib\site-packages\st

In [12]:
print(f"De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos {', '.join(tiempos)}")

De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos 6
